1) comment on catégoriser le taux d'influence: t-stat
2) is movie significant ? -> p-value
3) same release year for several movies -> keep the one that is the most rated (number of ratings)
4) keep categories of 5 : top 5, most 5 popular movies which are not significant, bottom 5
5) remove all movies that have less than 100 ratings (to be determined maybe we discard too many movies or too litte...)
6) remove all characters that have an order below the median (or mean) of the number of character in the movie.
7) Careful : the number of movies with significant impact on baby names may drop a lot due to the point 5 and 6
8) the point 5 and 6 must be done also for the global study and not only for the website interactive heading (TO BE DISCUSSED!!)

In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np

# New code

In [2]:
folder_processed_data_path = './processed_data/'
processed_website_data_folder = './processed_data/website/'

In [3]:
# import the name by movie
name_by_movie_df = pd.read_csv(os.path.join(folder_processed_data_path, 'name_by_movie_ordered_pvalue_10_5_df.csv'))
name_by_movie_df.set_index(['wiki_ID', 'char_words', 'gender'], inplace=True)
display(name_by_movie_df.sample(2))

# import the movie dataframe
movie_df = pd.read_csv(os.path.join(folder_processed_data_path, 'movie_df.csv'))
movie_df.set_index(['wiki_ID'], inplace=True)
display(movie_df.sample(2))

,,,order,p_value,slope_change,t_stat
wiki_ID,char_words,gender,,,,
14935392,Deuce,NaN,NaN,NaN,NaN,NaN
1522855,Haskell,M,11.0,0.894676,-0.000014,0.135485


,mov_name,year,month,revenue,numVotes,averageRating,poster_url
wiki_ID,,,,,,,
28745625,Batasari,1961,7.0,NaN,19,5.9,NaN
2095464,Mirch Masala,1985,NaN,NaN,1523,7.7,/j7vOnZuTnr420zdw3LDIq7froce.jpg


## Compute `movie_impact` dataframe

In [4]:
# given name by user
chosen_name = 'trinity'

First, let's compute the 5 years with the most change in the babies for the chosen name.

In [5]:
# Remove useless columns
name_by_movie_web = name_by_movie_df.reset_index().copy(deep=True)
name_by_movie_web.drop(columns=['order', 't_stat'], inplace=True)
display(name_by_movie_web.sample(2))

,wiki_ID,char_words,gender,p_value,slope_change
61689,4242732,Harvey,M,0.88454,-0.000082
157329,29309335,Kavi,M,NaN,NaN


In [6]:
# drop gender column
len_before = len(name_by_movie_web)

name_by_movie_web.reset_index(inplace=True, drop=True)
name_by_movie_web.drop_duplicates(subset=['wiki_ID', 'char_words'], keep='first', inplace=True)
name_by_movie_web.drop(columns=['gender'], inplace=True)
name_by_movie_web.set_index(['char_words', 'wiki_ID'], inplace=True)

len_after = len(name_by_movie_web)
print(f"length : {len_before} -> {len_after}")
print(f"Is the indexing of name_by_movie_web unique? {name_by_movie_web.index.is_unique}")
display(name_by_movie_web.sample(2))

length : 172906 -> 167984
Is the indexing of name_by_movie_web unique? True


,,p_value,slope_change
char_words,wiki_ID,,
Harper,23463534,0.997227,-8.334226e-07
Brink,6446044,NaN,NaN


In [7]:
# drop NaN values
len_before = len(name_by_movie_web)
name_by_movie_web.dropna(subset=['p_value'], inplace=True)
len_after = len(name_by_movie_web)
print(f"length : {len_before} -> {len_after}")

length : 167984 -> 116464


In [8]:
# Add the year of release of the movie to the name_by_movie_web dataframe

needed_movie_info = movie_df.reset_index()[['wiki_ID', 'year', 'averageRating', 'numVotes']].copy(deep=True)

len_before_merge = len(name_by_movie_web)
name_by_movie_with_info = name_by_movie_web.reset_index().merge(needed_movie_info, on='wiki_ID', how='left').copy(deep=True) # merge the release year into the name_by_movie_web dataframe
len_after_merge = len(name_by_movie_with_info)
print(f"length : {len_before_merge} -> {len_after_merge}")

name_by_movie_with_info.set_index(['char_words', 'wiki_ID'], inplace=True)
display(name_by_movie_with_info.sample(2))

length : 116464 -> 116464


,,p_value,slope_change,year,averageRating,numVotes
char_words,wiki_ID,,,,,
Clifford,952902,0.108558,-0.001012,2000,6.7,3831
Spencer,3599765,0.471155,-0.000429,1956,6.9,5767


In [9]:
# set the char_names in lowercase
name_by_movie_with_info.reset_index(inplace=True)
name_by_movie_with_info['char_words'] = name_by_movie_with_info['char_words'].str.lower()
name_by_movie_with_info.set_index(['char_words', 'wiki_ID'], inplace=True)
print(f"Length of name_by_movie_with_info : {len(name_by_movie_with_info)}")
display(name_by_movie_with_info.sample(2))

Length of name_by_movie_with_info : 116464


,,p_value,slope_change,year,averageRating,numVotes
char_words,wiki_ID,,,,,
sam,21678052,0.390860,0.000379,2010,5.1,256116
william,3662683,0.673892,-0.003222,2006,5.5,25100


In [10]:
# create the movie_impact dataframe
columns = ['name', 'status', 'movie_id']
movie_impact_df = pd.DataFrame(columns=columns)
display(movie_impact_df)

,name,status,movie_id


### Functions to compute TOP/BOTTOM/INSIGN

In [11]:
def compute_top_movies(name, chosen_name_movies_top_df, movie_impact_df):
    for i in range(5): # from top1 to top5
        if(not chosen_name_movies_top_df.empty):
            # get the movies release the year with the highest positive variation
            top_i_year = chosen_name_movies_top_df.iloc[-1].year.astype(int)
            top_i_year_chosen_name_movies = chosen_name_movies_top_df.query(f'year == {top_i_year}').copy(deep=True)

            # keep only the three most popular movies
            top_i_year_chosen_name_movies.sort_values(by=['numVotes'], ascending=False, inplace=True)
            top_i_3_chosen_name_movies = top_i_year_chosen_name_movies.iloc[:3].copy(deep=True)

            # add the the three popular movies to the movie_impact_df
            for index, row in top_i_3_chosen_name_movies.iterrows():
                movie_impact_df.loc[len(movie_impact_df)] = {'name': name, 'status': 't', 'movie_id': index}

            # remove the found movies from the chosen_name_movies_top_df to avoid picking them again for next iterations
            chosen_name_movies_top_df.drop(top_i_year_chosen_name_movies.index, inplace=True)

            # remove also the movies release in the frame [-5 years, +5 years] of the top1_year
            chosen_name_movies_top_df.query(f'year < {top_i_year - 5} or year > {top_i_year + 5}', inplace=True)

In [12]:
def compute_bottom_movies(name, chosen_name_movies_bottom_df, movie_impact_df):
    for i in range(5): # from bottom1 to bottom5
        if(not chosen_name_movies_bottom_df.empty):
            # get the movies release the year with the highest negative variation
            bottom_i_year = chosen_name_movies_bottom_df.iloc[-1].year.astype(int)
            bottom_i_year_chosen_name_movies = chosen_name_movies_bottom_df.query(f'year == {bottom_i_year}').copy(deep=True)

            # keep only the three most popular movies
            bottom_i_year_chosen_name_movies.sort_values(by=['numVotes'], ascending=False, inplace=True)
            bottom_i_3_chosen_name_movies = bottom_i_year_chosen_name_movies.iloc[:3].copy(deep=True)

            # add the the three popular movies to the movie_impact_df
            for index, row in bottom_i_3_chosen_name_movies.iterrows():
                movie_impact_df.loc[len(movie_impact_df)] = {'name': name, 'status': 'b', 'movie_id': index}

            # remove the found movies from the chosen_name_movies_bottom_df to avoid picking them again for next iterations
            chosen_name_movies_bottom_df.drop(bottom_i_year_chosen_name_movies.index, inplace=True)

            # remove also the movies release in the frame [-5 years, +5 years] of the bottom_i_year
            chosen_name_movies_bottom_df.query(f'year < {bottom_i_year - 5} or year > {bottom_i_year + 5}', inplace=True)

In [13]:
def compute_insign_movies(name, chosen_name_movies_insign_df, movie_impact_df):
    for i in range(5): # from insign1 to insign5
        if(not chosen_name_movies_insign_df.empty):
            # get the movies release the year with the highest rating and number of votes
            insign_i_year = chosen_name_movies_insign_df.iloc[-1].year.astype(int)
            insign_i_year_chosen_name_movies = chosen_name_movies_insign_df.query(f'year == {insign_i_year}').copy(deep=True)

            # keep only the three most popular movies
            insign_i_3_chosen_name_movies = insign_i_year_chosen_name_movies.iloc[:3].copy(deep=True)

            # add the the three popular movies to the movie_impact_df
            for index, row in insign_i_3_chosen_name_movies.iterrows():
                movie_impact_df.loc[len(movie_impact_df)] = {'name': name, 'status': 'i', 'movie_id': index}

            # remove the found movies from the chosen_name_movies_top_df to avoid picking them again for next iterations
            chosen_name_movies_insign_df.drop(insign_i_year_chosen_name_movies.index, inplace=True)

            # remove also the movies release in the frame [-5 years, +5 years] of the top1_year
            chosen_name_movies_insign_df.query(f'year < {insign_i_year - 5} or year > {insign_i_year + 5}', inplace=True)

### Special case for a single name

In [ ]:
# get the movie containing the chosen name
chosen_name_movies_df = name_by_movie_with_info.loc[chosen_name, :].copy(deep=True)
display(chosen_name_movies_df)

In [ ]:
chosen_name_movies_df.sort_values(by=['slope_change'], inplace=True)
chosen_name_movies_top_df = chosen_name_movies_df.query('(slope_change > 0) and (p_value < 0.1)').copy(deep=True)
display(chosen_name_movies_top_df)

Do the loop to do the computation for top1 to top5

In [ ]:
compute_top_movies(chosen_name, chosen_name_movies_top_df, movie_impact_df)

display(movie_impact_df)

In [ ]:
chosen_name_movies_bottom_df = chosen_name_movies_df.query('(slope_change <= 0) and (p_value < 0.1)').copy(deep=True)
chosen_name_movies_bottom_df.sort_values(by=['slope_change'], ascending=False, inplace=True)
display(chosen_name_movies_bottom_df)

In [ ]:
compute_bottom_movies(chosen_name, chosen_name_movies_bottom_df, movie_impact_df)

display(movie_impact_df)

In [ ]:
remaining_chosen_name_movies_df = pd.concat([chosen_name_movies_top_df, chosen_name_movies_bottom_df])
chosen_name_movies_insign_df = remaining_chosen_name_movies_df.query('p_value > 0.1').copy(deep=True)
chosen_name_movies_insign_df.sort_values(by=['averageRating', 'numVotes'], inplace=True)
display(chosen_name_movies_insign_df)

In [ ]:
compute_insign_movies(chosen_name, chosen_name_movies_insign_df, movie_impact_df)
display(movie_impact_df)

In [ ]:
# check if there movie duplicates in the movie_impact_df
print(f"number of duplicates in movie_impact_df : {movie_impact_df.duplicated(subset=['movie_id']).sum()}")

### Generalize for all the names

In [14]:
# get all the names in the name_by_movie_with_info dataframe
names = name_by_movie_with_info.index.get_level_values(0).unique().tolist()
print(names)
print(f"Number of names : {len(names)}")

['linda', 'henry', 'duke', 'williams', 'sheila', 'arthur', 'eldon', 'lewis', 'leon', 'hannibal', 'rick', 'rachael', 'tyrell', 'roy', 'bryant', 'sebastian', 'lamarr', 'van', 'bart', 'lyle', 'buddy', 'johnson', 'von', 'sam', 'howard', 'harriett', 'lili', 'jim', 'don', 'dorothy', 'hunter', 'frank', 'jeffrey', 'raymond', 'ben', 'paul', 'barbara', 'sandy', 'mike', 'lyndon', 'barry', 'kimberly', 'murray', 'nicole', 'andy', 'benny', 'gary', 'cassandra', 'joyce', 'jennifer', 'merrick', 'edward', 'campbell', 'william', 'king', 'princess', 'robert', 'stephen', 'wallace', 'stewart', 'bruce', 'malcolm', 'prince', 'isabelle', 'james', 'alexander', 'gordon', 'vicki', 'carl', 'bob', 'harvey', 'alfred', 'cooper', 'morgan', 'miles', 'robin', 'harriet', 'lady', 'max', 'charles', 'haven', 'ivy', 'wilson', 'grayson', 'nora', 'julie', 'madison', 'burton', 'chase', 'eric', 'roosevelt', 'jerry', 'thompson', 'leland', 'mary', 'anderson', 'herbert', 'young', 'susan', 'foster', 'walter', 'bertha', 'carter', 'th

In [15]:
iter = 0
for name in names:
    print(f"Number of name treated: {iter}", end='\r', flush=True)
    # get the movie containing the chosen name
    chosen_name_movies_df = name_by_movie_with_info.loc[name, :].copy(deep=True)

    # sort the movies by slope_change, get movies with significant p_value and positive slope_change and compute the top 5
    chosen_name_movies_df.sort_values(by=['slope_change'], inplace=True)
    chosen_name_movies_top_df = chosen_name_movies_df.query('(slope_change > 0) and (p_value < 0.1)').copy(deep=True)
    # display(chosen_name_movies_top_df)
    compute_top_movies(name, chosen_name_movies_top_df, movie_impact_df)

    # sort the movies by slope_change, get movies with significant p_value and negative slope_change and compute the bottom 5
    chosen_name_movies_bottom_df = chosen_name_movies_df.query('(slope_change <= 0) and (p_value < 0.1)').copy(deep=True)
    chosen_name_movies_bottom_df.sort_values(by=['slope_change'], ascending=False, inplace=True)
    compute_bottom_movies(name, chosen_name_movies_bottom_df, movie_impact_df)

    # get the remaining movies and filter to keep only the insignificant ones and compute the insign 5
    # remaining_chosen_name_movies_df = pd.concat([chosen_name_movies_top_df, chosen_name_movies_bottom_df])
    chosen_name_movies_insign_df = chosen_name_movies_df.query('p_value > 0.1').copy(deep=True)
    chosen_name_movies_insign_df.sort_values(by=['averageRating', 'numVotes'], inplace=True)
    compute_insign_movies(name, chosen_name_movies_insign_df, movie_impact_df)
    
    iter = iter + 1

In [36]:
# movie_impact_df.set_index(['name', 'status'], inplace=True)
display(movie_impact_df)

# Export DataFrame to a CSV file in the processed data folder
# movie_impact_df.reset_index(drop=True).sort_values(by=['name', 'status']).to_csv(os.path.join(processed_website_data_folder, 'movie_impact.csv'), index=False)

movie_id
name       status          
linda      t       14350910
           b        4070671
           b        9956825
           b        2212664
           b       28288095
...                     ...
serene     i       36544941
kathrine   i       36546325
zelma      i       36598217
beauregard b       36699915
carlina    i       37196243

[27854 rows x 1 columns]

### Import `movie_impact` dataframe to avoid timeconsuming computation

In [ ]:
# import the movie_impact_df to avoid the time-consuming computation of movie_impact_df
imported_movie_impact_df = pd.read_csv(os.path.join(processed_website_data_folder, 'movie_impact.csv'))
imported_movie_impact_df.set_index(['name', 'status'], inplace=True)
display(imported_movie_impact_df)

In [ ]:
# Check for NaN values in the name column
contains_nan = imported_movie_impact_df.reset_index()['name'].isna().any()

if contains_nan:
    print("The `name` column contains NaN values.")
else:
    print("The `name` column does not contain NaN values.")

In [ ]:
# drop NaN values in the name column
print(f"Length of imported_movie_impact_df before dropping NaN values : {len(imported_movie_impact_df)}")
imported_movie_impact_df.reset_index(inplace=True)
imported_movie_impact_df.dropna(subset=['name'], inplace=True)
imported_movie_impact_df.set_index(['name', 'status'], inplace=True)
print(f"Length of imported_movie_impact_df after dropping NaN values : {len(imported_movie_impact_df)}")

display(imported_movie_impact_df)
movie_impact_df = imported_movie_impact_df.copy(deep=True)

## Compute `name_per_year` dataframe

In [18]:
# import the babynames dataframe
name_per_year = pd.read_csv(os.path.join(folder_processed_data_path, 'baby_name_df.csv'))
name_per_year.drop(columns='number', inplace=True)
name_per_year.set_index(['name', 'year'], inplace=True)
print(f"Is the indexing of name_per_year unique? {name_per_year.index.is_unique}")
display(name_per_year.sample(2))
print(f"Length of the name_per_year_df : {len(name_per_year)}")

Is the indexing of name_per_year unique? True


,,percentage
name,year,
Burley,1924,0.001596
Alondria,1985,0.000196


Length of the name_per_year_df : 1903290


In [19]:
# round the percentage values to reduce the size of the future csv file for the web
name_per_year['percentage'] = name_per_year['percentage'].round(6)

# set the names in lowercases
name_per_year.reset_index(inplace=True)
name_per_year['name'] = name_per_year['name'].str.lower()
name_per_year.set_index(['name', 'year'], inplace=True)

display(name_per_year.sample(2))
print(f"Length of the name_per_year_df : {len(name_per_year)}")

,,percentage
name,year,
shar,1969,0.000230
genaro,1941,0.001272


Length of the name_per_year_df : 1903290


In [23]:
# filter the name_per_year_df to keep only the names in the movie_impact_df
name_per_year_filtered = name_per_year.loc[movie_impact_df['name'].unique().tolist(), :].copy(deep=True)
name_per_year = name_per_year_filtered.copy(deep=True)

In [24]:
# Let's sort the dataframe name_per_year to anticipate the ploting
name_per_year.sort_values(by=['name', 'year'], inplace=True)
display(name_per_year.head(15))

percentage
name  year            
aadam 1987    0.000139
      1988    0.000135
      1993    0.000186
      1994    0.000161
      1995    0.000164
      1996    0.000137
      1997    0.000138
      1998    0.000218
      1999    0.000135
      2000    0.000159
      2002    0.000294
      2003    0.000263
      2004    0.000236
      2005    0.000156
      2006    0.000228

Let's check if the names present in name_per_year dataframe are also present in the name_by_movie dataframe

In [25]:
# Identify names in name_per_year not present in name_by_movie
values_only_in_name_per_year = name_per_year.reset_index()[~name_per_year.reset_index()['name'].isin(name_by_movie_with_info.reset_index()['char_words'])]['name'].unique()

print(f"Number of unique names in name_per_year  = {len(name_per_year.reset_index()['name'].unique())}")

# Display the result
print("Values in name_per_year but not present in name_by_movie:")
print(values_only_in_name_per_year)
print(f"Number of names missing  = {len(values_only_in_name_per_year)}")

Number of unique names in name_per_year  = 6034
Values in name_per_year but not present in name_by_movie:
[]
Number of names missing  = 0


In [26]:
# Identify names in name_by_movie not present in name_per_year
values_only_in_name_by_movie = name_by_movie_with_info.reset_index()[~name_by_movie_with_info.reset_index()['char_words'].isin(name_per_year.reset_index()['name'])]['char_words'].unique()

print(f"Number of unique names in name_by_movie_with_info  = {len(name_by_movie_with_info.reset_index()['char_words'].unique())}")

# Display the result
print("Values in name_by_movie_with_info but not present in name_per_year:")
print(values_only_in_name_by_movie)
print(f"Number of names missing  = {len(values_only_in_name_by_movie)}")

Number of unique names in name_by_movie_with_info  = 6034
Values in name_by_movie_with_info but not present in name_per_year:
[]
Number of names missing  = 0


Compute the number of names in the three dataframes in order to compare

In [30]:
# compute how many unique names are there in the dataframes to compare
names_in_name_per_year = name_per_year.index.get_level_values(0).unique().tolist()
print(f"Number of unique names in name_per_year = {len(names_in_name_per_year)}")
names_in_movie_impact = movie_impact_df.reset_index()['name'].unique().tolist()
print(f"Number of unique names in movie_impact_df = {len(names_in_movie_impact)}")
names_in_name_by_movie_with_info = name_by_movie_with_info.reset_index()['char_words'].unique().tolist()
print(f"Number of unique names in name_by_movie_with_info = {len(names_in_name_by_movie_with_info)}")

Number of unique names in name_per_year = 6034
Number of unique names in movie_impact_df = 6034
Number of unique names in name_by_movie_with_info = 6034


## Compute `movies` dataframe
This dataset contains the information relative to the movie given its `wiki_ID`.

In [32]:
print(f"Number of movies in movie_impact_df : {len(movie_impact_df['movie_id'].unique())}")
# keep only the movies in the movie_df that are in the movie_impact_df
simplified_movie_df = movie_df.loc[movie_impact_df['movie_id'].unique().tolist(), :].copy(deep=True)
print(f"Number of movies kept in movie_df: {len(simplified_movie_df)}")
display(simplified_movie_df.sample(2))

Number of movies in movie_impact_df : 13953
Number of movies kept in movie_df: 13953


,mov_name,year,month,revenue,numVotes,averageRating,poster_url
wiki_ID,,,,,,,
5980485,Polish Wedding,1998,1.0,600294.0,3401,5.5,/36De1BxBBzXyJ1eYVQlIoTUIwQm.jpg
27988012,Vaanam,2011,4.0,NaN,1914,7.3,/kE9kN2iFaM77suP00RHYMOoqUos.jpg


## Export the three datasets

In [37]:
# Export movie_impact.csv
display(movie_impact_df.reset_index().sample(2))
movie_impact_df.reset_index().to_csv(os.path.join(processed_website_data_folder, 'movie_impacts.csv'), index=False)

# Export name_per_year.csv
display(name_per_year.reset_index().sample(2))
name_per_year.reset_index().to_csv(os.path.join(processed_website_data_folder, 'name_per_year.csv'), index=False)

# Export movie.csv
display(simplified_movie_df.reset_index().sample(2))
simplified_movie_df.reset_index().to_csv(os.path.join(processed_website_data_folder, 'movies.csv'), index=False)

,name,status,movie_id
13987,anastasia,i,16305665
24731,leilani,t,3753182


,name,year,percentage
218193,hedy,1943,0.003047
104918,colette,1974,0.008256


,wiki_ID,mov_name,year,month,revenue,numVotes,averageRating,poster_url
5323,23520462,Deep Gold,2011,4.0,NaN,363,3.2,/4eCQoTJ3QXDxBlXChlc7cF909WO.jpg
9107,24517581,The Naked Face,1984,6.0,NaN,1438,5.5,/4RHE44sgmW44pd6Klf4cfGVOm0C.jpg


# Old code

In [ ]:
folder_processed_data_path = './processed_data/'

# import the name_by_movie dataframe
name_by_movie_df = pd.read_csv(os.path.join(folder_processed_data_path, 'name_by_movie_ordered_df.csv'))
name_by_movie_df.set_index(['wiki_ID', 'char_words', 'gender'], inplace=True)
display(name_by_movie_df.sample(2))

# import the movie dataframe
movie_df = pd.read_csv(os.path.join(folder_processed_data_path, 'movie_df.csv'))
movie_df.set_index(['wiki_ID'], inplace=True)
display(movie_df.sample(2))

# import the baby names dataframe
baby_name_df = pd.read_csv(os.path.join(folder_processed_data_path, 'baby_name_df.csv'))
baby_name_df.set_index(['name', 'year'], inplace=True)
display(baby_name_df.sample(2))

# create dataframe containing the release year of each movie
release_year_df = movie_df[['year', 'numVotes']].copy(deep=True)
display(release_year_df.sample(2))

# import the name_by_movie with the slope difference
name_by_movie_slope_diff_df = pd.read_csv(os.path.join(folder_processed_data_path, 'name_by_movie_ordered_pvalue_10_5_df.csv'))
name_by_movie_slope_diff_df.set_index(['wiki_ID', 'char_words', 'gender'], inplace=True)
display(name_by_movie_slope_diff_df.sample(2))

# Data processing for the website
In this notebook, we filter the `name_by_movie` dataframe in order to remove the movie character too little known to have had an impact on the baby names. We will assess the popularity of the charcter based on the number of IMDB ratings of the movie and whether its role was important by using the `order` attribute.

In addition, an issue related to the limitation of the data is assessed here. The problem is that for two movies released the same year that has the same name for one of their character, we can't tell which of them had an impact on the name given to babies. To assess this problem, we will keep only the most popular movie based on the number of IMDB ratings.

### Filtering of `name_by_movie` : number of ratings

First, let's remove the character names of movies with less than 100 IMDB ratings.

In [ ]:
name_by_movie_df.reset_index(inplace=True)
movie_df.reset_index(inplace=True)

name_by_movie_df_merged = name_by_movie_df.merge(release_year_df, left_on='wiki_ID', right_on='wiki_ID', how = 'left').copy(deep=True)
print(f"length of the initial dataframe : {len(name_by_movie_df)}")
print(f"length of the merged dataframe : {len(name_by_movie_df_merged)}")
display(name_by_movie_df_merged.sample(2))

In [ ]:
name_by_movie_df_merged_filter_nbratings = name_by_movie_df_merged[name_by_movie_df_merged['numVotes'] >= 1000].copy(deep=True)
print(f"length of the initial dataframe : {len(name_by_movie_df)}")
print(f"length of the filtered dataframe : {len(name_by_movie_df_merged_filter_nbratings)}")

### Filtering of `name_by_movie`  : importance of the role

Now, lets remove the character names with an minor role in the movie. We will keep only the characters with a order higher or equal to the median of the set of order in the movie. First, let's compute the number of order for each movies.

In [ ]:
# is it normal that not all the movies in movies_df have at least one character in name_by_movie_df ? TODO
# display(movie_df[movie_df['wiki_ID'] == 844398])
# display(name_by_movie_df[name_by_movie_df['wiki_ID'] == 844398])

In [ ]:
name_by_movie_merged_groupby = name_by_movie_df_merged_filter_nbratings.groupby(['wiki_ID'])
name_by_movie_merged_nunique = name_by_movie_merged_groupby['order'].nunique()

# check for a specific movie
test_movie_id = 617063

for movie_id, group in name_by_movie_merged_groupby:
    if(movie_id == (test_movie_id,)):
        print('movie_id', movie_id)
        print('group', group)

print(f"number of unique order = {name_by_movie_merged_nunique[test_movie_id]}")
name_by_movie_merged_nunique = name_by_movie_merged_nunique.to_frame()
name_by_movie_merged_nunique.rename(columns={"order": "nb_order"}, inplace=True)
display(name_by_movie_merged_nunique)

In [ ]:
name_by_movie_merged_with_nunique = name_by_movie_df_merged_filter_nbratings.merge(name_by_movie_merged_nunique, on='wiki_ID', how='left')
display(name_by_movie_merged_with_nunique.sample(2))

In [ ]:
# filtering
name_by_movie_merged_important_role = name_by_movie_merged_with_nunique[name_by_movie_merged_with_nunique['order'] <= name_by_movie_merged_with_nunique['nb_order']].copy(deep=True)
name_by_movie_merged_important_role.drop(columns=['order', 'nb_order'], inplace=True)
print(f"length of the initial dataframe : {len(name_by_movie_df)}")
print(f"length of the filtered dataframe : {len(name_by_movie_merged_important_role)}")
display(name_by_movie_merged_important_role.sample(2))

### Filtering  `name_by_movie` : keep three most famous character of the year only
In a given year, several movies may have been released with the same name for one of their character. This is problematic because it is impossible to know which of the movie had an impact on the baby naming of the given name (if it had any). To solve this problem, we keep three most famous movies (based on the number of rating) for each year and name in the dataset. 

First, we need to remove the gender attribute which is annoying because some of the characters are twice in the dataset once for each genders.

In [ ]:
# drop gender column
len_before = len(name_by_movie_merged_important_role)
name_by_movie_merged_important_role.drop_duplicates(subset=['wiki_ID', 'char_words'], keep='first', inplace=True)
name_by_movie_merged_important_role.drop(columns=['gender'], inplace=True)
len_after = len(name_by_movie_merged_important_role)
print(f"length before : {len_before}")
print(f"length after : {len_after}")

Now, we can filter the `name_by_movie` dataframe in order to keep only the three most popular movies for each year and name

In [ ]:
name_by_movie_merged_three_characters_only = name_by_movie_merged_important_role.sort_values(by='numVotes', ascending=False).groupby(['year', 'char_words']).head(3).reset_index(drop=True)

display(name_by_movie_merged_three_characters_only)

Now, let's make a first test to see if the filtering is correct for a specific case: the name Daniel, in 2001.

In [ ]:
# check before
test_word = 'Daniel'
test_year = 2001
test_df = name_by_movie_merged_important_role[name_by_movie_merged_important_role['char_words'] == test_word]
test_df = test_df[test_df['year'] == test_year]
display(test_df)

In [ ]:
# check after
test_df = name_by_movie_merged_three_characters_only[name_by_movie_merged_three_characters_only['char_words'] == test_word]
test_df = test_df[test_df['year'] == test_year]
display(test_df)

In [ ]:
print(f"length of the initial dataframe : {len(name_by_movie_df)}")
print(f"length of the filtered dataframe : {len(name_by_movie_merged_three_characters_only)}")

### Create the custom `web_name_by_movie_df`
Finally, we create the dataframe that will be used in the website. To do so, we add information about the movie on the `name_by_movie` dataframe created until now.

In [ ]:
# display(name_by_movie_merged_three_characters_only.sample(2))
# display(movie_df)

# Add movie info to the dataframe
temp_merge_df = movie_df[['wiki_ID', 'mov_name', 'averageRating', 'poster_url']].copy(deep=True)
web_name_by_movie_df = name_by_movie_merged_three_characters_only.merge(temp_merge_df, on='wiki_ID', how='left').copy(deep=True)

# Reorder the columns
desired_columns_order = ['char_words', 'wiki_ID', 'mov_name', 'year', 'averageRating', 'numVotes', 'poster_url']
web_name_by_movie_df = web_name_by_movie_df[desired_columns_order].copy(deep=True)
web_name_by_movie_df.set_index(['char_words', 'wiki_ID'], inplace=True)

# Check if the indexing is unique
print(f"Is the indexing of web_name_by_movie_df unique ? {web_name_by_movie_df.index.is_unique}")
display(web_name_by_movie_df.sample(2))

In [ ]:
# Specific check
web_name_by_movie_df.loc['Elizabeth'][web_name_by_movie_df.loc['Elizabeth']['year'] == 1940]

### Filtering of `baby_name_df`
The baby_name_df is too large to be loaded on the website. We will drop the column `number` and remove the names with too little years where there is data.

In [ ]:
display(baby_name_df.head(2))
web_baby_name_df = baby_name_df[['percentage']].copy(deep=True)
display(web_baby_name_df.head(2))
web_baby_name_count = web_baby_name_df.groupby(['name']).count()
web_baby_name_count.rename(columns={"percentage": "count"}, inplace=True)
display(web_baby_name_count.head(2))
plt.hist(web_baby_name_count['count'], bins=100)
plt.show()

# merging
web_baby_name_df.reset_index(inplace=True)
web_baby_name_count.reset_index(inplace=True)
display(web_baby_name_df.head(2))
display(web_baby_name_count.head(2))
web_name_by_movie_df_merged = web_baby_name_df.merge(web_baby_name_count, on='name', how='left').copy(deep=True)
display(web_name_by_movie_df_merged.head(2))

# filtering
print(f"size of the dataframe before filtering : {len(baby_name_df)}")
print(f"size of the dataframe before filtering : {len(web_name_by_movie_df_merged)}")
web_baby_name_df = web_name_by_movie_df_merged[web_name_by_movie_df_merged['count'] >= 60].copy(deep=True)
web_baby_name_df.drop(columns=['count'], inplace=True)
web_baby_name_df.set_index(['name', 'year'], inplace=True)
display(web_baby_name_df.head(2))
print(f"size of the dataframe after filtering : {len(web_baby_name_df)}")

In [ ]:
# Let's filter the dataframe web_baby_name_df to anticipate the ploting
web_baby_name_df.sort_values(by=['year'], inplace=True)

### Creation of the TOP5/5NULLFAMOUS/BOTTOM5
We need also a dataframe containing for each name in the `baby_name` dataset, 10 years where a variation of the baby names are significant (5 with the largest positive variation and 5 with the largest negative variation) as well as 5 popular movies where the variation is not significant.

In [ ]:
display(name_by_movie_slope_diff_df.sample(2))

Let's remove the gender because we won't consider it

In [ ]:
# drop gender column
len_before = len(name_by_movie_slope_diff_df)
name_by_movie_slope_diff_without_gender_df = name_by_movie_slope_diff_df.copy(deep=True)
name_by_movie_slope_diff_without_gender_df.reset_index(inplace=True)
name_by_movie_slope_diff_without_gender_df.drop_duplicates(subset=['wiki_ID', 'char_words'], keep='first', inplace=True)
name_by_movie_slope_diff_without_gender_df.drop(columns=['gender'], inplace=True)
len_after = len(name_by_movie_slope_diff_without_gender_df)
print(f"length before : {len_before}")
print(f"length after : {len_after}")
display(name_by_movie_slope_diff_without_gender_df.sample(2))

In [ ]:
# drop useless columns
name_by_movie_with_slope_only = name_by_movie_slope_diff_without_gender_df.drop(columns=['order', 'p_value', 't_stat']).copy(deep=True)
display(name_by_movie_with_slope_only.sample(2))

Now let's tackle the computing of the 5 TOP and 5 BOTTOM years with the most variation per names

In [ ]:
movie_id_and_year_df = movie_df[['wiki_ID', 'year']].copy(deep=True)
display(movie_id_and_year_df.sample(2))

In [ ]:
# merge the release year into the name_by_movie_with_slope_only dataframe
name_by_movie_with_slope_and_year = name_by_movie_with_slope_only.merge(movie_id_and_year_df, on='wiki_ID', how='left').copy(deep=True)
display(name_by_movie_with_slope_and_year.sample(2))

# check for a specific name and year
name_by_movie_with_slope_and_year.set_index(['char_words', 'year'], inplace=True)
display(name_by_movie_with_slope_and_year.loc['Elizabeth', 2004]) # ok, everything is fine
name_by_movie_with_slope_and_year.reset_index(inplace=True)

In [ ]:
display(name_by_movie_with_slope_and_year.head(2))
name_by_year_with_slop_change = name_by_movie_with_slope_and_year.drop_duplicates(subset=['char_words', 'year'], keep='first').copy(deep=True)
name_by_year_with_slop_change.drop(columns=['wiki_ID'], inplace=True)
name_by_year_with_slop_change.dropna(subset='slope_change', inplace=True)
display(name_by_year_with_slop_change.head(2))

In [ ]:
# Find the top and bottom 5 years for each name
top_5_years = name_by_year_with_slop_change.groupby('char_words').apply(lambda x: x.nlargest(5, 'slope_change')).reset_index(drop=True).copy(deep=True)
bottom_5_years = name_by_year_with_slop_change.groupby('char_words').apply(lambda x: x.nsmallest(5, 'slope_change')).reset_index(drop=True).copy(deep=True)

# Add a column indicating whether it is a top or bottom year
top_5_years['top_or_bottom'] = 'top'
bottom_5_years['top_or_bottom'] = 'bottom'

# Concatenate the two DataFrames
web_top_bottom_df = pd.concat([top_5_years, bottom_5_years], ignore_index=True).copy(deep=True)

# Drop the column used for sorting and set the index columns
web_top_bottom_df.set_index(['char_words', 'top_or_bottom'], inplace=True)

display(web_top_bottom_df)

We have an issue when the are less than 5 years with a positive slope and when there are less than 5 years with a negative slope. The next cell solve this problem.

In [ ]:
display(web_top_bottom_df)

In [ ]:
display(web_top_bottom_df.loc['Maxime',:]) ## There should be only positive value in for the top years and only negative values for the bottom years

# To solve this problem :
web_top_bottom_df_no_index = web_top_bottom_df.reset_index().copy(deep=True)
web_top_bottom_filtered_df = web_top_bottom_df_no_index[~((web_top_bottom_df_no_index['slope_change'] < 0) & (web_top_bottom_df_no_index['top_or_bottom'] == 'top')) & ~((web_top_bottom_df_no_index['slope_change'] > 0) & (web_top_bottom_df_no_index['top_or_bottom'] == 'bottom'))].copy(deep=True)
web_top_bottom_filtered_df.set_index(['char_words', 'top_or_bottom'], inplace=True)

display(web_top_bottom_filtered_df.loc['Maxime',:]) ## There should be only positive value in for the top years and only negative values for the bottom years

In [ ]:
web_top_bottom_df = web_top_bottom_filtered_df.copy(deep=True)

In [ ]:
# check TOP 5 and BOTTOM 5 computation with a specific name
name_by_year_with_slop_change.reset_index(inplace=True)
name_by_year_with_slop_change.set_index(['char_words', 'year'], inplace=True)
display(name_by_year_with_slop_change)
display(name_by_year_with_slop_change.loc['Elizabeth', :])

import plotly.express as px

# Create a scatter plot using Plotly Express
fig = px.scatter(name_by_year_with_slop_change.loc['Elizabeth', :].reset_index(), x='year', y='slope_change', title='Interactive Scatter Plot', labels={'X': 'X-axis', 'Y': 'Y-axis'})

# Add cursor tooltip
fig.update_layout(hovermode='closest')

# Show the plot
fig.show()

display(web_top_bottom_df.loc['Elizabeth', :])

In [ ]:
# Check passed, we can drop the column slope_change
web_top_bottom_df.drop(columns=['slope_change'], inplace=True)

Now, for each name in the `web_name_by_movie` dataframe, we gather all the movies that have this name for one of its character and that are not significant. Then we take the first 5 of this list and add it into a new dataframe called `5_famous_movies_but_unsignificant_by_name`

We reuse the dataframe `name_by_movie_slope_diff_without_gender_df` computed above.

In [ ]:
display(name_by_movie_slope_diff_without_gender_df.head())
print(f"Length before : {len(name_by_movie_slope_diff_without_gender_df)}")

# first we can remove all the rows where the p_value is a NaN
name_by_movie_slope_diff_without_gender_filtered = name_by_movie_slope_diff_without_gender_df.copy(deep=True)
name_by_movie_slope_diff_without_gender_filtered.dropna(subset=['p_value'], inplace=True)
# we can remove the columsn we don't need
name_by_movie_slope_diff_without_gender_filtered.drop(columns=['order', 'slope_change', 't_stat'], inplace=True)
display(name_by_movie_slope_diff_without_gender_filtered.head())
print(f"Length after : {len(name_by_movie_slope_diff_without_gender_filtered)}")

print(f"Length before : {len(name_by_movie_slope_diff_without_gender_df)}")
# keep only non-significent rows
name_by_movie_non_significent = name_by_movie_slope_diff_without_gender_filtered[name_by_movie_slope_diff_without_gender_filtered['p_value'] >= 0.1].copy(deep=True)
name_by_movie_non_significent.drop(columns=['p_value'], inplace=True)
display(name_by_movie_non_significent.head())
print(f"Length after : {len(name_by_movie_non_significent)}")

In [ ]:
# add the attribute 'numVotes' from the movie_df dataframe with a merge
temp_merge_df = movie_df[['wiki_ID', 'numVotes']].copy(deep=True)
display(temp_merge_df.head())
name_by_movie_non_significent_with_numvotes = name_by_movie_non_significent.merge(temp_merge_df, on='wiki_ID', how='left').copy(deep=True)
display(name_by_movie_non_significent_with_numvotes.head())
print(f"length of name_by_movie_non_significent_with_numvotes : {len(name_by_movie_non_significent_with_numvotes)}")

In [ ]:
# Sorting the DataFrame by 'name' and 'scores' in descending order
# df_sorted = name_by_movie_non_significent_with_numvotes.sort_values(by=['char_words', 'numVotes'], ascending=[True, False])
name_by_movie_non_significent_with_numvotes_sorted = name_by_movie_non_significent_with_numvotes.sort_values(by='numVotes', ascending=False).copy(deep=True)
display(name_by_movie_non_significent_with_numvotes_sorted.head())

# Extracting the top 5 rows for each 'name'
web_top_famous_but_insi = name_by_movie_non_significent_with_numvotes_sorted.groupby('char_words').head(5).copy(deep=True)
web_top_famous_but_insi.set_index(['char_words'], inplace=True)
web_top_famous_but_insi.drop(columns=['numVotes'], inplace=True)
display(web_top_famous_but_insi.head())

### Export the data

In [ ]:
processed_website_data_folder = './processed_data/website/'

display(web_name_by_movie_df.reset_index().head())
# Export DataFrame to a CSV file in the processed data folder
web_name_by_movie_df.reset_index().to_csv(os.path.join(processed_website_data_folder, 'web_name_by_movie_df.csv'), index=False)

display(web_baby_name_df.reset_index().head())
# Export DataFrame to a CSV file in the processed data folder
web_baby_name_df.reset_index().to_csv(os.path.join(processed_website_data_folder, 'web_baby_name_df.csv'), index=False)

display(web_top_bottom_df.reset_index().head())
# Export DataFrame to a CSV file in the processed data folder
web_top_bottom_df.reset_index().to_csv(os.path.join(processed_website_data_folder, 'web_top_bottom_df.csv'), index=False)

display(web_top_famous_but_insi.reset_index().head())
# Export DataFrame to a CSV file in the processed data folder
web_top_famous_but_insi.reset_index().to_csv(os.path.join(processed_website_data_folder, 'web_top_famous_but_insi.csv'), index=False)

# Test the ploting

In [ ]:
# To be set by the user
chosen_name = 'Elizabeth'

web_baby_name_df.reset_index(inplace=True)
web_baby_name_df.set_index(['name'], inplace=True)

x_values = web_baby_name_df.loc[chosen_name]['year'].values
y_values = web_baby_name_df.loc[chosen_name]['percentage'].values

# Ploting
plt.plot(x_values, y_values)
plt.xlabel('Years')
plt.ylabel('Percentage of total births')
plt.title(f'Name "{chosen_name}"')

In [ ]:
# Get the top 5 years from the web data
top_5_years = web_top_bottom_df.loc[chosen_name, 'top'].values[:,0]
print(f"top_5_years = {top_5_years}")
data_top_5_years = web_name_by_movie_df[(web_name_by_movie_df['char_words'] == chosen_name) & (web_name_by_movie_df['year'].isin(top_5_years))].copy(deep=True)
display(data_top_5_years)

In [ ]:
# Get the bottom 5 years from the web data
bottom_5_years = web_top_bottom_df.loc[chosen_name, 'bottom'].values[:,0]
print(f"bottom_5_years = {bottom_5_years}")
data_bottom_5_years = web_name_by_movie_df[(web_name_by_movie_df['char_words'] == chosen_name) & (web_name_by_movie_df['year'].isin(bottom_5_years))].copy(deep=True)
display(data_bottom_5_years)

# # only three movies for the chosen name 'Thomas' ? instead of 3x5=15 ? check the top 5 years seperately
# display(web_name_by_movie_df[(web_name_by_movie_df['char_words'] == chosen_name) & (web_name_by_movie_df['year'] == bottom_5_years[0])])
# display(web_name_by_movie_df[(web_name_by_movie_df['char_words'] == chosen_name) & (web_name_by_movie_df['year'] == bottom_5_years[1])])
# display(web_name_by_movie_df[(web_name_by_movie_df['char_words'] == chosen_name) & (web_name_by_movie_df['year'] == bottom_5_years[2])])
# display(web_name_by_movie_df[(web_name_by_movie_df['char_words'] == chosen_name) & (web_name_by_movie_df['year'] == bottom_5_years[3])])
# display(web_name_by_movie_df[(web_name_by_movie_df['char_words'] == chosen_name) & (web_name_by_movie_df['year'] == bottom_5_years[4])])

In [ ]:
# Get the top 5 famous but insignificant movies from the web data
id_5_famous_movies_but_insignificant = web_top_famous_but_insi.loc[chosen_name].values[:,0]
print(f"id_5_famous_movies_but_insignificant = {id_5_famous_movies_but_insignificant}")

web_name_by_movie_df.reset_index(inplace=True)
data_5_famous_movies_but_insignificant = web_name_by_movie_df[(web_name_by_movie_df['char_words'] == chosen_name) & (web_name_by_movie_df['wiki_ID'].isin(id_5_famous_movies_but_insignificant))]
display(data_5_famous_movies_but_insignificant)